# Some Useful Links:

### Pytorch Broadcasting Semantics:
- https://docs.python.org/3/library/operator.html


# Lectures 7 and 8: Building Autograd engine from scratch

## Learning outcomes

1. Understanding Automated Differentiation Engines at a foundation level
2. Operator Overloading in OOP
3. Graph Representation
4. Graph Traversal

In [2]:
from random import Random
from math import sqrt
from uuid import uuid4
import numpy as np

In [3]:
# Code Copied from previous lectures
SEED = 5

random_gen = Random(x = SEED)
def generate_points(N = 1000):
    lst_x, lst_y = [], []
    for _ in range(N):
        lst_x.append(random_gen.uniform(a = 0, b = 1))
    for _ in range(N):
        lst_y.append(random_gen.uniform(a = 0, b = 1))
    return lst_x, lst_y


def loss(x_p, y_p, batch_x, batch_y):
    return ( 1 / len(batch_x)) * sum( [sqrt( (x_i - x_p)**2 + (y_i - y_p)**2)
                                      for x_i, y_i in zip(batch_x, batch_y)])

def calc_grad(x_p, y_p, batch_x, batch_y):
    sum_x , sum_y = 0., 0.

    for x_i, y_i in zip(batch_x, batch_y):
        inv_sqrt = ((x_i - x_p) ** 2 + (y_i  - y_p) ** 2) ** (-0.5)
        sum_x += inv_sqrt * (x_i - x_p)
        sum_y += inv_sqrt * (y_i - y_p)

    return -sum_x / len(batch_x), -sum_y / len(batch_x)

In [4]:
data_x, data_y = generate_points(N=100)
x_p, y_p = 0.3, 0.3
grad_x, grad_y = calc_grad(x_p, y_p, data_x, data_y)
cur_loss = loss(x_p, y_p, data_x, data_y)
print(f"Closed form: gradient for x_p {grad_x}, gradient for y_p {grad_y}")
print(f"Closed_form: loss = {cur_loss}")

Closed form: gradient for x_p -0.28245211967555384, gradient for y_p -0.5339538940488842
Closed_form: loss = 0.5031831004924388


## Pytorch as an example of Autograd engines

In [5]:
import torch

DELTA = 0.001
pnt = torch.tensor([0.3, 0.3])
pnt.requires_grad = True
pnt.retain_grad()
data_x, data_y = generate_points(N = 1)
data = torch.tensor([data_x, data_y])
data = data.t()
print(data.shape, pnt.shape)

for epoch in range(2):
  loss_torch = torch.mean(torch.sqrt(((data - pnt)**2).sum(dim = 1)))
  print(f"torch_loss: {loss_torch}")
  loss_torch.backward()
  with torch.no_grad():
    print(pnt)
    pnt -= DELTA*pnt.grad.data
    print(f"torch grad: {pnt.grad.data}")
    pnt.grad.zero_()

torch.Size([1, 2]) torch.Size([2])
torch_loss: 0.39435747265815735
tensor([0.3000, 0.3000], requires_grad=True)
torch grad: tensor([ 0.2053, -0.9787])
torch_loss: 0.39335745573043823
tensor([0.2998, 0.3010], requires_grad=True)
torch grad: tensor([ 0.2053, -0.9787])


## Building Autograd from scratch

In [6]:
class comp_node:
  def __init__(self, val, children = [], op = "assign"):
    self.val = val
    self.children = children
    self.op = op
    self.grad = 0
    self.backward_prop = lambda : None
    self.identity = uuid4()


  def __to_comp_node(self, obj):
    if not isinstance(obj, comp_node):
      return comp_node(val = obj)
    else:
      return obj


  def __sub__(self, other):
    other = self.__to_comp_node(other)
    out = comp_node(val = self.val - other.val, children = [self, other] , op = "sub")
    def _backward_prop():
      self.grad += out.grad * 1
      other.grad += out.grad * (-1)

    out.backward_prop = _backward_prop
    return out


  def __rsub__(self, other):
    other = self.__to_comp_node(other)

    return other - self


  def __add__(self, other):
    other = self.__to_comp_node(other)
    out = comp_node(val = self.val + other.val, children = [self, other], op = "add")

    def _backward_prop():
      self.grad += out.grad * 1
      other.grad += out.grad * 1

    out.backward_prop = _backward_prop
    return out


  def __radd__(self, other):
    other = self.__to_comp_node(other)
    return other + self


  def __mul__(self, other):
    other = self.__to_comp_node(other)
    out = comp_node(val = self.val * other.val, children = [self, other], op = "mult")

    def _backward_prop():
      self.grad += out.grad * (other.val)
      other.grad += out.grad * (self.val)
    out.backward_prop = _backward_prop
    return out


  def __rmul__(self, other):
    other = self.__to_comp_node(other)
    return other * self



  def __pow__(self, exponent):
    if not isinstance(exponent, (int, float)):
      raise ValueError("Unsupported types")
    out = comp_node(val = self.val**exponent, children = [self], op = f"power {exponent}")
    def _backward_prop():
      self.grad += out.grad * (exponent * self.val**(exponent - 1))
    out.backward_prop = _backward_prop
    return out


  def __eq__(self, other):
    return self.val == other.val


  def __repr__(self):
    return f"op: {self.op} | val: {self.val: .4f} | childern: {len(self.children)} | grad: {self.grad}"

######################## Start New Code ########################
  # A unique key for each node (to be accessed from the set or the dictionary)
  def __hash__(self):
    return int(self.identity)

  def topo_sort(self, collect_edges=False):
    res=[]
    visited= set()
    def visit(node):
      if node not in visited:
        visited.add(node)
        for child in node.children:
            visit(child)
        res.append(node)
    visit(self)
    return res

  def backward(self):
    nodes = self.topo_sort()
    self.grad = 1
    for node in reversed(nodes):
      node.backward_prop()
######################## End New Code ########################


assert comp_node(val = 5).val == 5, "Assignment failed"
assert (comp_node(val = 5) - 3).val == 2
assert (2 - comp_node(val = 5)).val == -3
assert (comp_node(val = 5)**2).val == 25
assert (comp_node(val = 5)**2) == comp_node(val = 25)
assert (comp_node(val = 5) + 3).val == 8
assert (2 + comp_node(val = 5)).val == 7
assert (comp_node(val = 5) * 3).val == 15
assert (2 * comp_node(val = 5)).val == 10

In [7]:
data_x, data_y = generate_points(N=1)
x_p, y_p = comp_node(val = 0.3), comp_node(val = 0.3)


def loss_graph(x_p, y_p, data_x, data_y):
  I_x, I_y = x_p - data_x, y_p - data_y
  g_x, g_y = I_x**2, I_y**2
  M = g_x + g_y
  l = M ** 0.5
  return l, [l, M, g_x, g_y, I_x, I_y, x_p, y_p]


l, rev_topo_order = loss_graph(x_p, y_p, data_x[0], data_y[0])
rev_topo_order[0].grad = 1

for i, node in enumerate(rev_topo_order):
  node.backward_prop()
  print(i, node)

0 op: power 0.5 | val:  0.4197 | childern: 1 | grad: 1
1 op: add | val:  0.1761 | childern: 2 | grad: 1.1914656577245382
2 op: power 2 | val:  0.1095 | childern: 1 | grad: 1.1914656577245382
3 op: power 2 | val:  0.0666 | childern: 1 | grad: 1.1914656577245382
4 op: sub | val: -0.3309 | childern: 2 | grad: -0.7884289557292833
5 op: sub | val:  0.2581 | childern: 2 | grad: 0.6151258259637875
6 op: assign | val:  0.3000 | childern: 0 | grad: -0.7884289557292833
7 op: assign | val:  0.3000 | childern: 0 | grad: 0.6151258259637875


In [ ]:
#################### Start New Code ####################
data_x, data_y = generate_points(N=100)
x_p, y_p = comp_node(val = 0.3), comp_node(val = 0.3)

def loss_graph(x_p, y_p, data_x, data_y):
  loss=0
  for x_i, y_i in zip(data_x, data_y):
    I_x, I_y = x_p - x_i, y_p - y_i
    g_x, g_y = I_x**2, I_y**2
    M = g_x + g_y
    l = M ** 0.5
    loss += l
  return (1/len(data_x)) * loss

curr_loss = loss_graph(x_p, y_p, data_x, data_y)
curr_loss.backward()

print(x_p.grad, y_p.grad)
#from pprint import pprint
#pprint(list(reversed(curr_loss.topo_sort())))
print(x_p.grad, y_p.grad)
#################### End New Code ####################

-0.4495361695632005 -0.30993795041828925
-0.4495361695632005 -0.30993795041828925
